Run the cells below to load the data into a database file that can be operated on with SQL

In [2]:
%load_ext sql
%sql sqlite:///Data/airbnb.db

'Connected: @Data/airbnb.db'

In [6]:
import pandas as pd
import os

def load_csv_to_df(fn):
    
    with open(fn) as data_file:
        df = pd.read_csv(fn)
        print("Loaded data frame. ")
        return df

In [7]:
csv_files = ['detailedListings', 'detailedReviews', 'neighbourhoods']
dfs = {}

for f in csv_files:
    fn = "./Data/" + f + ".csv"
    dfs[f] = load_csv_to_df(fn)
    print("Loaded file " + f) 

dfs['detailedListings'] = dfs['detailedListings'].drop(columns=
                                                       ['amenities', 'host_picture_url', 'host_thumbnail_url', 
                                                        'host_about', 'host_url','picture_url', 'house_rules', 
                                                        'interaction', 'access', 'transit', 'notes',
                                                        'neighborhood_overview', 'experiences_offered', 
                                                        'description', 'space', 'summary'])

Loaded data frame. 
Loaded file Calendar


/Users/mli/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (61,62,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Loaded data frame. 
Loaded file detailedListings
Loaded data frame. 
Loaded file detailedReviews
Loaded data frame. 
Loaded file reviews
Loaded data frame. 
Loaded file listings
Loaded data frame. 
Loaded file neighbourhoods


In [9]:
%%sql
drop table if exists Calendar;
drop table if exists detailedListings;
drop table if exists detailedReviews;
drop table if exists reviews;
drop table if exists listings;
drop table if exists neighbourhoods;

 * sqlite:///Data/airbnb.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [10]:
import sqlite3
conn = sqlite3.connect('./Data/airbnb.db')

for k,v in dfs.items():
    if 'id' in v or 'index' in v: 
        v.to_sql(name = k, con=conn, if_exists='append', index_label=None)
        print("Created and loaded table = ", k)
    else:
        v.to_sql(name = k, con=conn, if_exists='append', index_label='id')
        print("Created and loaded table = ", k)

Created and loaded table =  Calendar
Created and loaded table =  detailedListings
Created and loaded table =  detailedReviews
Created and loaded table =  reviews
Created and loaded table =  listings
Created and loaded table =  neighbourhoods


In [57]:
%%sql
CREATE TABLE ListingsAndReviews as
select listing_id, review_date, comments, name, neighbourhood, neighbourhood_cleansed as accurate_neighbourhood, 
zipcode, property_type, room_type, bathrooms, beds, accommodates, price, number_of_reviews, number_of_reviews_ltm, 
review_scores_rating, review_scores_location, reviews_per_month
from
(select listing_id, date as review_date, comments from detailedReviews) dr JOIN
detailedListings as dl
ON dl.id = dr.listing_id;

 * sqlite:///Data/airbnb.db
Done.


[]

Cells below are just used to test what tables might look like

In [2]:
%%sql
select * from ListingsAndReviews
order by accurate_neighbourhood
limit 5;

 * sqlite:///Data/airbnb.db
Done.


listing_id,review_date,comments,name,neighbourhood,accurate_neighbourhood,zipcode,property_type,room_type,bathrooms,beds,accommodates,price,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_location,reviews_per_month
182177,2011-09-04,"This was my first time using airbnb and it was awesome. Christian and his family are very accomodating and they responded quickly whenever my or I had a question. The area was nice, quiet, and transportation was easily accessible. The house had all the major necessites you would expect it to have. Would definitely stay there again!",A PRIVATE FLAT / APARTMENT- $SPECIAL$,Baychester,Allerton,10469,Guest suite,Entire home/apt,1.0,2.0,8,$125.00,295,35,93.0,10.0,2.93
182177,2011-09-05,Hey! This place is great! Easy to find and get to - better price and facilities than absolutely everything a reasonable person can find in Manhatten. And a very generous host in Christian. I'd recommend this place to anyone!,A PRIVATE FLAT / APARTMENT- $SPECIAL$,Baychester,Allerton,10469,Guest suite,Entire home/apt,1.0,2.0,8,$125.00,295,35,93.0,10.0,2.93
182177,2011-09-12,"Our experience with Christian was wonderful. He and his family were very accomodating. The apartment was wonderful, perfect for a small family and would work well for a group of friends too. Parking is excellent and the backyard is a special treat. The house was just as it is pictured, very fresh and clean. The neigborhood is fantastic, whether you want Chinese, Italian or simply some milk- all of it is within 2 minutes walking. My only concern about leaving this review is that this apartment will always be booked and I won't get to stay again. This will be our first choice any time we are headed to NYC. Thank you Christian for making our week in NYC a great one! Barb and Darrell Turner",A PRIVATE FLAT / APARTMENT- $SPECIAL$,Baychester,Allerton,10469,Guest suite,Entire home/apt,1.0,2.0,8,$125.00,295,35,93.0,10.0,2.93
182177,2011-09-19,I love this place! (which is why I have returned!). And I will for sure return if I visit the Bronx again! The Palacios' eagerness to understand a travellers needs is really amazing! If anyone tells you that NY is gonna eat you alive it's because you haven't had this place as your base! Compared to paying four times the price for a manhattan-place with less than half the size or comfort this is the best deal you can find! From the door you have less than an hour to the absolute most southern point of manhattan.,A PRIVATE FLAT / APARTMENT- $SPECIAL$,Baychester,Allerton,10469,Guest suite,Entire home/apt,1.0,2.0,8,$125.00,295,35,93.0,10.0,2.93
182177,2011-09-28,"the house was an amazing place, christian and his family were so amazingly accommodating. We had troubles and they made them all go away!! thanks so much Jacky and Francisco, you were so helpful and nice. Perhaps the only issue is its distance from downtown manhattan where, as young students, there was most of the action.",A PRIVATE FLAT / APARTMENT- $SPECIAL$,Baychester,Allerton,10469,Guest suite,Entire home/apt,1.0,2.0,8,$125.00,295,35,93.0,10.0,2.93


In [5]:
%%sql
select comments from ListingsAndReviews
where accurate_neighbourhood='Allerton'
limit 5;

 * sqlite:///Data/airbnb.db
Done.


comments
"This was my first time using airbnb and it was awesome. Christian and his family are very accomodating and they responded quickly whenever my or I had a question. The area was nice, quiet, and transportation was easily accessible. The house had all the major necessites you would expect it to have. Would definitely stay there again!"
Hey! This place is great! Easy to find and get to - better price and facilities than absolutely everything a reasonable person can find in Manhatten. And a very generous host in Christian. I'd recommend this place to anyone!
"Our experience with Christian was wonderful. He and his family were very accomodating. The apartment was wonderful, perfect for a small family and would work well for a group of friends too. Parking is excellent and the backyard is a special treat. The house was just as it is pictured, very fresh and clean. The neigborhood is fantastic, whether you want Chinese, Italian or simply some milk- all of it is within 2 minutes walking. My only concern about leaving this review is that this apartment will always be booked and I won't get to stay again. This will be our first choice any time we are headed to NYC. Thank you Christian for making our week in NYC a great one! Barb and Darrell Turner"
I love this place! (which is why I have returned!). And I will for sure return if I visit the Bronx again! The Palacios' eagerness to understand a travellers needs is really amazing! If anyone tells you that NY is gonna eat you alive it's because you haven't had this place as your base! Compared to paying four times the price for a manhattan-place with less than half the size or comfort this is the best deal you can find! From the door you have less than an hour to the absolute most southern point of manhattan.
"the house was an amazing place, christian and his family were so amazingly accommodating. We had troubles and they made them all go away!! thanks so much Jacky and Francisco, you were so helpful and nice. Perhaps the only issue is its distance from downtown manhattan where, as young students, there was most of the action."


In [22]:
%%sql
select accurate_neighbourhood, AVG(review_scores_rating) as average_review 
from ListingsAndReviews
where accurate_neighbourhood = "Allerton"
order by average_review desc;

 * sqlite:///Data/airbnb.db
Done.


accurate_neighbourhood,average_review
Allerton,92.44398530762167
